# Factorización LU

In [1]:
format long
fmt = format


















fmt = long

















In [2]:
graphics_toolkit ("qt")

In [3]:
printf('Octave version: %s.', version)


Octave version: 8.4.0.

## `LUFactSimple.m`

In [4]:
function [L, U, err] = LUFactSimple(A)
    % LU factorization of a square matrix.
    %
    % Input
    %   A(1:n,1:n) : the matrix to be factorized
    %
    % Output
    %   L(1:n,1:n), U(1:n,1:n) : the factors in A = LU, if Gaussian
    %                            elimination proceeds to completion
    %   err : = 0 if Gaussian elimination proceeds to completion
    %         = 1 if a zero pivot is encountered
    n = size(A, 1);
    U = A;
    L = eye(n);
    err = 0;

    for k = 1:n - 1

        for j = k + 1:n

            if abs(U(k, k)) > eps(U(k, k))
                L(j, k) = U(j, k) / U(k, k);
            else
                err = 1;
                return;
            end

            for t = k:n
                U(j, t) = U(j, t) - L(j, k) * U(k, t);
            end

        end

    end

end

### Prueba de `LUFactSimple.m`

In [5]:
clearvars

n = 3;
A = rand(n, n)

tic
[L, U, err] = LUFactSimple(A)
toc

L * U





A =

   0.193612137176367   0.499369802177195   0.908209774240669
   0.317067200786767   0.207857489980856   0.713665423418793
   0.698598660699526   0.695351076208063   0.233330543552219




L =

   1.000000000000000                   0                   0
   1.637641138674797   1.000000000000000                   0
   3.608237948756031   1.814129662579689   1.000000000000000

U =

   0.193612137176367   0.499369802177195   0.908209774240669
                   0  -0.609931041476413  -0.773656265424277
                   0                   0  -1.640193649547306

err = 0

Elapsed time is 1.01829 seconds.


ans =

   0.193612137176367   0.499369802177195   0.908209774240669
   0.317067200786767   0.207857489980856   0.713665423418793
   0.698598660699526   0.695351076208063   0.233330543552219



## `TriDiagonal.m`

In [6]:
function [x, err] = TriDiagonal(a, b, c, f)
    % Solution of a linear system of equations with a tridiagonal
    % matrix.
    %
    %   a(k) x(k-1) + b(k) x(k) + c(k) x(k+1) = f(k)
    %
    % with a(1) = c(n) = 0.
    %
    % Input
    %   a(1:n), b(1:n), c(1:n) : the coefficients of the system
    %                            matrix
    %   f(1:n) : the right hand side vector
    %
    % Output
    %   x(1:n) : the solution to the linear system of equations, if
    %            no division by zero occurs
    %   err : = 0, if no division by zero occurs
    %         = 1, if division by zero is encountered
    n = length(f);
    alpha = zeros(n, 1);
    beta = zeros(n, 1);
    err = 0;

    if abs(b(1)) > eps(b(1))
        alpha(1) = -c(1) / b(1);
        beta(1) = f(1) / b(1);
    else
        err = 1;
        return;
    end

    for k = 2:n
        denominator = a(k) * alpha(k - 1) + b(k);

        if abs(denominator) > eps(denominator)
            alpha(k) =- c(k) / denominator;
            beta(k) = (f(k) - a(k) * beta(k - 1)) / denominator;
        else
            err = 1;
            return;
        end

    end

    if abs(a(n) * alpha(n - 1) + b(n)) > eps(b(n))
        x(n) = (f(n) - a(n) * beta(n - 1)) / (a(n) * alpha(n - 1) + b(n));
    else
        err = 1;
        return;
    end

    for k = n - 1:-1:1
        x(k) = alpha(k) * x(k + 1) + beta(k);
    end

end

### Prueba de `TriDiagonal.m`

In [7]:
clearvars

n = 5;
a = rand(n, 1)
b = rand(n, 1)
c = rand(n, 1)
f = rand(n, 1)

tic
[x, err] = TriDiagonal(a, b, c, f)
toc

lhs = @(k) a(k) * x(k - 1) + b(k) * x(k) + c(k) * x(k + 1);
rhs = @(k) f(k);

printf('k \t lhs(k) \t rhs(k)\n')
for k=2:n-1
    printf('k = %d\t%f\t%f\n', k, lhs(k), rhs(k))
end





a =

   0.636994198604416
   0.847628580625249
   0.830596812390029
   0.137209160813121
   0.271579289594824


b =

   6.239302577908457e-02
   3.051475418691983e-01
   2.946517965221521e-01
   3.428149522861001e-02
   2.466118515895778e-01


c =

   4.845915093875397e-02
   1.869529131428281e-01
   6.967676521714327e-01
   6.364847078076046e-01
   3.673940710587243e-01


f =

   0.460765394631403
   0.710663654615507
   0.612400957331049
   0.933122678758500
   0.839896151906647




x =

 Columns 1 through 3:

   1.612305785714049e+00   7.432419908363340e+00  -1.564007442347846e+01

 Columns 4 and 5:

  -1.367123311929063e+00   4.911274628295483e+00

err = 0

Elapsed time is 1.0193 seconds.





k 	 lhs(k) 	 rhs(k)



k = 2	0.710664	0.710664
k = 3	0.612401	0.612401
k = 4	0.933123	0.933123


## `CyclicallyTriDiagonal.m`

In [8]:
function [x, err] = CyclicallyTriDiagonal(a, b, c, f)
    % Solution of a cyclically tridiagonal linear system of equations:
    %
    %   b(1) x(  1) + c(1) x(  2) + a(1) x(  n) = f(1),
    %   a(k) x(k-1) + b(k) x(  k) + c(k) x(k+1) = f(k),
    %   c(n) x(  1) + a(n) x(n-1) + b(n) x(  n) = f(n).
    %
    % Input
    %   a(1:n), b(1:n), c(1:n) : the coefficients of the system
    %                            matrix
    %   f(1:n) : the right hand side vector
    %
    % Output
    %   x(1:n) : the solution to the linear system of equations, if
    %            no division by zero is encountered
    %   err : = 0, if division by zero does not occur
    %         = 1, if division by zero occurs
    n = length(f);
    err = 0;
    newa = a(2:n);
    newa(1) = 0;
    newb = b(2:n);
    newc = c(2:n);
    newc(n - 1) = 0;
    newf = f(2:n);
    [u, err] = TriDiagonal(newa, newb, newc, newf);

    if err == 1
        return;
    end

    newf = zeros(n - 1, 1);
    newf(1) =- a(2);
    newf(n - 1) = -c(n);
    [v, err] = TriDiagonal(newa, newb, newc, newf);

    if err == 1
        return;
    end

    x = zeros(n, 1);
    denominator = b(1) + c(1) * v(1) + a(1) * v(n - 1);

    if abs(denominator) > eps(denominator)
        x(1) = (f(1) - c(1) * u(1) - a(1) * u(n - 1)) / denominator;
    else
        err = 1;
        return;
    end

    for k = 2:n
        x(k) = u(k - 1) + x(1) * v(k - 1);
    end

end

### Prueba de `CyclicallyTriDiagonal.m`

In [9]:
clearvars

n = 5;
a = rand(n, 1)
b = rand(n, 1)
c = rand(n, 1)
f = rand(n, 1)

tic
[x, err] = CyclicallyTriDiagonal(a, b, c, f)
toc

lhs = @(k) a(k) * x(k - 1) + b(k) * x(k) + c(k) * x(k + 1);
rhs = @(k) f(k);

printf('k \t lhs(k) \t rhs(k)\n')
for k=2:n-1
    printf('k = %d\t%f\t%f\n', k, lhs(k), rhs(k))
end





a =

   1.414083563427496e-01
   9.752258025107673e-01
   3.561982011488354e-01
   9.414403295440188e-04
   6.287125769385733e-01


b =

   1.265977170750334e-01
   2.301173865942713e-01
   7.125007200021898e-01
   1.112285752215281e-02
   4.482838222314717e-01


c =

   0.328292707375058
   0.999541322140132
   0.346582128024455
   0.300830406027820
   0.914262366132487


f =

   0.719155167738610
   0.104680722124815
   0.296373157176504
   0.569612435901048
   0.866943191939615




x =

   4.070253288502485e-02
   1.358466670650224e+00
  -2.477338682381545e-01
  -3.173755035736903e-02
   1.895415700611055e+00

err = 0

Elapsed time is 1.02519 seconds.





k 	 lhs(k) 	 rhs(k)



k = 2	0.104681	0.104681
k = 3	0.296373	0.296373
k = 4	0.569612	0.569612


## `LUFactEfficient.m`

In [10]:
function [Afact, swaps, err] = LUFactEfficient(A)
    % Efficient implementation of LU factorization with partial
    % pivoting.
    %
    % Input
    %   A(1:n,1:n) : the matrix to be factorized into A = P'LU
    %
    % Output
    %   Afact(1:n,1:n) : a matrix containing the matrices L and U
    %                    below and above the diagonal, respectively
    %   swaps : the indices that indicate the permutations
    %           (row swaps)
    %   err : = 0 if no the factorization finished successfully
    %         = 1 if a division by zero was encountered
    n = size(A, 1);
    swaps = 1:n;
    err = 0;

    for k = 1:n - 1
        i = k;

        for t = k:n

            if abs(A(t, k)) > abs(A(i, k))
                i = t;
            end

        end

        tt = swaps(k);
        swaps(k) = swaps(i);
        swaps(i) = tt;

        if abs(A(swaps(k), k)) <= eps(A(swaps(k), k))
            err = 1;
            return;
        end

        for i = k + 1:n
            xmult = A(swaps(i), k) / A(swaps(k), k);
            A(swaps(i), k) = xmult;

            for j = k + 1:n
                A(swaps(i), j) = A(swaps(i), j) - xmult * A(swaps(k), j);
            end

        end

    end

    Afact = A;
end

### Prueba de `LUFactEfficient.m`

In [11]:
clearvars

n = 3;
A = rand(n, n)

tic
[Afact, swaps, err] = LUFactEfficient(A)
toc





A =

   3.921636211102963e-02   5.291734926634668e-01   2.158836781230512e-01
   7.383221404543326e-01   8.193107863939713e-01   2.087101727891225e-01
   3.933294728653925e-01   2.626520498863091e-01   8.847704106641421e-01




Afact =

   5.311551687573329e-02   4.856553767622875e-01   2.047979294181334e-01
   7.383221404543326e-01   8.193107863939713e-01   2.087101727891225e-01
   5.327342244177508e-01  -3.579139751714543e-01   8.468833996601612e-01

swaps =

   2   1   3

err = 0

Elapsed time is 1.01942 seconds.


## `CholeskyDecomposition.m`

In [12]:
function [R, err] = CholeskyDecomposition(A)
    % Choleksy factorization of the HPD matrix A:
    %   A = R'*R
    %
    % Input:
    %   A(1:n,1:n) : an HPD matrix
    %
    % Output:
    %   R(1:n,1:n) : The upper triangular matrix satisfying A = R'*R
    %   err : = 0 if the decomposition finished successfully
    %         = 1 if an error occurred
    err = 0;
    n = size(A, 1);
    R = zeros(n, n);
    R(1, 1) = sqrt(A(1, 1));

    for i = 2:n

        for j = 1:i - 1
            sum = A(i, j);

            for k = 1:j - 1
                sum = sum - R(k, i) * conj(R(k, j));
            end

            if abs(R(j, j)) > eps(R(j, j))
                R(j, i) = sum / R(j, j);
            else
                err = 1;
                return;
            end

        end

        sum = A(i, i);

        for k = 1:i - 1
            sum = sum - R(k, i) * conj(R(k, i));
        end

        R(i, i) = sqrt(sum);
    end

end

### Prueba de `CholeskyDecomposition.m`

In [13]:
clearvars

A = [1 1/2; 1/2 1]

tic
[R, err] = CholeskyDecomposition(A)
toc

R' * R




A =

   1.000000000000000   0.500000000000000
   0.500000000000000   1.000000000000000




R =

   1.000000000000000   0.500000000000000
                   0   0.866025403784439

err = 0

Elapsed time is 1.02022 seconds.


ans =

   1.000000000000000   0.500000000000000
   0.500000000000000   1.000000000000000



## `Solve.m`

In [14]:
function [x, err] = Solve(A, f)
    % Solves the system Ax = f.
    %
    % Input
    %   A(1:n,1:n) : the coefficient matrix
    %   f(1:n) : the RHS vector
    %
    % Output:
    %   x(1:n) : the solution vector
    %   err : = 0 if the solution was found successfully
    %         = 1 if an error occurred during the process
    n = length(f);
    err = 0;
    [AA, swaps, err] = LUFactEfficient(A);

    if err == 1
        return
    end

    for k = 1:n - 1

        for i = k + 1:n
            f(swaps(i)) = f(swaps(i)) - AA(swaps(i), k) ...
                *f(swaps(k));
        end

    end

    if abs(AA(swaps(n), n)) > eps(AA(swaps(n), n))
        x(n) = f(swaps(n)) / AA(swaps(n), n);
    else
        err = 1;
        return;
    end

    for i = n - 1:-1:1
        xsum = f(swaps(i));

        for j = i + 1:n
            xsum = xsum - AA(swaps(i), j) * x(j);
        end

        if abs(AA(swaps(i), i)) > eps(AA(swaps(i), i))
            x(i) = xsum / AA(swaps(i), i);
        else
            err = 1;
            return;
        end

    end

end

### Prueba de `Solve.m`

In [15]:
n = 10
A = rand(n, n);
f = rand(n, 1)

tic
[x, err] = Solve(A, f)
toc

A * x'


n = 10


f =

   5.619586163763024e-02
   8.523537582243459e-01
   5.949006547200777e-01
   4.562344695828212e-01
   7.499564929158672e-01
   1.581871076235011e-01
   8.354579987229185e-01
   1.634936375606203e-01
   7.106603426216054e-01
   6.428692656801144e-01




x =

 Columns 1 through 4:

  -1.680723872241640   0.504707081039916   4.605031163666300  -3.317463779213350

 Columns 5 through 8:

   6.781643370296245  -6.971087280175980  -3.110126294986005  -3.531408364121731

 Columns 9 and 10:

   4.408539357670882   0.362007875749561

err = 0

Elapsed time is 1.03188 seconds.


ans =

   5.619586163763035e-02
   8.523537582243454e-01
   5.949006547200804e-01
   4.562344695828191e-01
   7.499564929158686e-01
   1.581871076235011e-01
   8.354579987229158e-01
   1.634936375606202e-01
   7.106603426216009e-01
   6.428692656801100e-01

